# Exploring, cleaning and enriching data

# 1. Exploring

In [5]:
import numpy as np
import pandas as pd
import src.clean_utils as cu
import requests
from bs4 import BeautifulSoup
import re

### In this dataframe the information that we want to get is how are NY AirBnB vitits distribute by neighbourhood.

In [6]:
# Importing Data, we know the topic but, What tipe of datas do we have?
Abb_NY = pd.read_csv("data/AB_NYC_2019.csv",encoding = "ISO-8859-1")

In [7]:
Abb_NY.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [8]:
#We can se than we have a very big Dataset but, would it be too dirty?
Abb_NY.shape

(48895, 16)

In [9]:
Abb_NY.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [10]:
# It seems I found a very clean dataset. Where only 2 columns  have over 20% of nulls datas but they don`t affect to our study.
perc_null_col = Abb_NY.isnull().sum().apply(lambda x: x/Abb_NY.shape[0]).sort_values(ascending=False)
perc_null_col

reviews_per_month                 0.205583
last_review                       0.205583
host_name                         0.000429
name                              0.000327
availability_365                  0.000000
calculated_host_listings_count    0.000000
number_of_reviews                 0.000000
minimum_nights                    0.000000
price                             0.000000
room_type                         0.000000
longitude                         0.000000
latitude                          0.000000
neighbourhood                     0.000000
neighbourhood_group               0.000000
host_id                           0.000000
id                                0.000000
dtype: float64

In [11]:
# And what about our rows? there are a few with missing datas, but as we saw this datas are from "reviews" column and do not affect us in our study.
perc_null_row=Abb_NY.isnull().sum(axis=1).apply(lambda x: x/Abb_NY.shape[1]).sort_values(ascending=False)
perc_null_row

16071    0.1875
27777    0.1875
38992    0.1875
6605     0.1875
2854     0.1875
          ...  
29611    0.0000
29610    0.0000
29608    0.0000
29607    0.0000
0        0.0000
Length: 48895, dtype: float64

In [12]:
# There are not duplicate visits:
Abb_NY.duplicated().sum()

0

In [13]:
# It looks like we have both numerical and categorical data
Abb_NY_dtype=Abb_NY.dtypes
Abb_NY_dtype

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

After this review and exploring of data, we can conclude we have a very good dataframe, with clean data, quantity to be analyze and some columns we can drop and will  not affect our conclusions.

# 2. Cleaning de Data

In [14]:
# We remenber the columns we have in order to drop those we dont need
Abb_NY.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [15]:
# We drop those we would not use
drop_cols = ["id","name","host_id","host_name","latitude", "longitude","reviews_per_month","calculated_host_listings_count","availability_365"]
Abb_NY_clean = Abb_NY.drop(drop_cols, axis =1)
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review
0,Brooklyn,Kensington,Private room,149,1,9,2018-10-19
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [16]:
# I would like to focuss my attention on those "normal years", and by normal I mean out of the Great recession.
Abb_NY_clean["last_review"].value_counts(dropna=False)

NaN           10052
2019-06-23     1413
2019-07-01     1359
2019-06-30     1341
2019-06-24      875
              ...  
2015-04-12        1
2014-10-28        1
2014-05-02        1
2014-12-22        1
2016-11-22        1
Name: last_review, Length: 1765, dtype: int64

In [17]:
#Add a column with clean year from date
Abb_NY_clean["year"] = Abb_NY_clean.last_review.dropna().apply(cu.extract_year)

In [18]:
Abb_NY_clean.head()

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,last_review,year
0,Brooklyn,Kensington,Private room,149,1,9,2018-10-19,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019-05-21,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019-07-05,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018-11-19,2018.0


In [19]:
#Count our years
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
NaN       10052
2018.0     6050
2017.0     3205
2016.0     2707
2015.0     1393
2014.0      199
2013.0       48
2012.0       25
2011.0        7
Name: year, dtype: int64

In [20]:
# Drop date column that is no longer usefull
drop_cols = ["last_review"]
Abb_NY_clean = Abb_NY_clean.drop(drop_cols, axis =1)
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [21]:
# So we will focuss on those years after 2015
Abb_NY_clean["year"] = Abb_NY_clean[Abb_NY_clean["year"]>2015]["year"]
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
2,Manhattan,Harlem,Private room,150,3,0,NaN
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
...,...,...,...,...,...,...,...
48890,Brooklyn,Bedford-Stuyvesant,Private room,70,2,0,NaN
48891,Brooklyn,Bushwick,Private room,40,4,0,NaN
48892,Manhattan,Harlem,Entire home/apt,115,10,0,NaN
48893,Manhattan,Hell's Kitchen,Shared room,55,1,0,NaN


In [22]:
#Drop nulls out from > 2015
Abb_NY_clean = Abb_NY_clean[Abb_NY_clean["year"].notnull()]
Abb_NY_clean

,neighbourhood_group,neighbourhood,room_type,price,minimum_nights,number_of_reviews,year
0,Brooklyn,Kensington,Private room,149,1,9,2018.0
1,Manhattan,Midtown,Entire home/apt,225,1,45,2019.0
3,Brooklyn,Clinton Hill,Entire home/apt,89,1,270,2019.0
4,Manhattan,East Harlem,Entire home/apt,80,10,9,2018.0
5,Manhattan,Murray Hill,Entire home/apt,200,3,74,2019.0
...,...,...,...,...,...,...,...
48782,Manhattan,Upper East Side,Private room,129,1,1,2019.0
48790,Queens,Flushing,Private room,45,1,1,2019.0
48799,Staten Island,Great Kills,Private room,235,1,1,2019.0
48805,Bronx,Mott Haven,Entire home/apt,100,1,2,2019.0


In [23]:
#And we finally have our clean dataframe
#Wtih clean years
Abb_NY_clean["year"].value_counts(dropna=False)

2019.0    25209
2018.0     6050
2017.0     3205
2016.0     2707
Name: year, dtype: int64

In [24]:
#And clean neighbourhood
Abb_NY_clean["neighbourhood_group"].value_counts(dropna=False)

Manhattan        15834
Brooklyn         15702
Queens            4457
Bronx              869
Staten Island      309
Name: neighbourhood_group, dtype: int64

In [25]:
#And we export our datas to be able to use them while looking for visualization
Abb_NY_clean.to_csv("output/Abb_NY_output.csv", index = False)

# 3. Enriching de Data

### In this dataframe the information that we want to get is how are NY monuments are distribute by neighbourhood.

In [26]:
#We import our URL, were we have seen is the important information for enrich our study. 
url_NYmonuments = "https://en.wikipedia.org/wiki/List_of_buildings,_sites,_and_monuments_in_New_York_City"

In [27]:
response = requests.get(url_NYmonuments)

In [28]:
response

<Response [200]>

In [29]:
soup = BeautifulSoup(response.content)

In [30]:
#We call for the important info we need:
NYmonuments_tags = soup.find_all(
    name="div", 
    class_="div-col"
)

In [31]:
#And filter it to get it as clean as possible:
NYmonuments_tags_li = soup.find_all("li")

In [32]:
NYmonuments_tags_text_li = [monument.text for monument in NYmonuments_tags_li]
NYmonuments_tags_text_li[2]

"America's Response Monument (Manhattan)"

In [33]:
len(NYmonuments_tags_text_li)

307

In [34]:
# We drop the lines we dont need:
NYmonuments_tags_text_li2 = NYmonuments_tags_text_li[0:105]
NYmonuments_tags_text_li2[:5]

['American Museum of Natural History (Manhattan)\nRose Center for Earth and Space',
 'Rose Center for Earth and Space',
 "America's Response Monument (Manhattan)",
 'Apollo Theater (Manhattan)',
 'Bank of America Tower (Manhattan)']

In [35]:
#And make it more visual by using a dataframe:
df_NYmonuments = pd.DataFrame(NYmonuments_tags_text_li2)
df_NYmonuments.columns = ["monuments_neighbourhood"]
df_NYmonuments

,monuments_neighbourhood
0,American Museum of Natural History (Manhattan)...
1,Rose Center for Earth and Space
2,America's Response Monument (Manhattan)
3,Apollo Theater (Manhattan)
4,Bank of America Tower (Manhattan)
...,...
100,Woodlawn Cemetery (Bronx)
101,Woolworth Building (Manhattan)
102,World Trade Center site (Manhattan)
103,World Financial Center (Manhattan)


In [36]:
# We clean our colums, split then and take out the ()
df_NYmonuments[["monuments","neighbourhood"]] = df_NYmonuments.monuments_neighbourhood.str.split("\(|\)", expand=True).iloc[:,[0,1]]
df_NYmonuments

,monuments_neighbourhood,monuments,neighbourhood
0,American Museum of Natural History (Manhattan)...,American Museum of Natural History,Manhattan
1,Rose Center for Earth and Space,Rose Center for Earth and Space,None
2,America's Response Monument (Manhattan),America's Response Monument,Manhattan
3,Apollo Theater (Manhattan),Apollo Theater,Manhattan
4,Bank of America Tower (Manhattan),Bank of America Tower,Manhattan
...,...,...,...
100,Woodlawn Cemetery (Bronx),Woodlawn Cemetery,Bronx
101,Woolworth Building (Manhattan),Woolworth Building,Manhattan
102,World Trade Center site (Manhattan),World Trade Center site,Manhattan
103,World Financial Center (Manhattan),World Financial Center,Manhattan


In [37]:
# Drop the columns not needed:
drop_cols = ["monuments_neighbourhood"]
df_NYmonuments = df_NYmonuments.drop(drop_cols, axis =1)
df_NYmonuments

,monuments,neighbourhood
0,American Museum of Natural History,Manhattan
1,Rose Center for Earth and Space,None
2,America's Response Monument,Manhattan
3,Apollo Theater,Manhattan
4,Bank of America Tower,Manhattan
...,...,...
100,Woodlawn Cemetery,Bronx
101,Woolworth Building,Manhattan
102,World Trade Center site,Manhattan
103,World Financial Center,Manhattan


In [38]:
df_NYmonuments["neighbourhood"].value_counts(dropna=False)

Manhattan                                       64
NaN                                             12
Brooklyn                                         8
Bronx                                            6
Queens                                           5
connects Manhattan and New Jersey                3
connects Manhattan and Brooklyn                  2
formerly known as the New York State Theater     1
Staten Island                                    1
connects Brooklyn and Manhattan                  1
The Bronx                                        1
Queens; demolished 2009                          1
Name: neighbourhood, dtype: int64

In [39]:
# And drop the info that is not usefull:
df_NYmonuments = df_NYmonuments[df_NYmonuments["neighbourhood"].notnull()]
df_NYmonuments

,monuments,neighbourhood
0,American Museum of Natural History,Manhattan
2,America's Response Monument,Manhattan
3,Apollo Theater,Manhattan
4,Bank of America Tower,Manhattan
5,Battery Park,Manhattan
...,...,...
100,Woodlawn Cemetery,Bronx
101,Woolworth Building,Manhattan
102,World Trade Center site,Manhattan
103,World Financial Center,Manhattan


In [40]:
#use a funtion in order to clean our column neighbourhood and make it iqual to our reviews dataframe.
df_NYmonuments["neighbourhood"] = df_NYmonuments["neighbourhood"].apply(cu.cleaning)
df_NYmonuments

<ipython-input-40-36042f0c3889>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_NYmonuments["neighbourhood"] = df_NYmonuments["neighbourhood"].apply(cu.cleaning)


,monuments,neighbourhood
0,American Museum of Natural History,manhattan
2,America's Response Monument,manhattan
3,Apollo Theater,manhattan
4,Bank of America Tower,manhattan
5,Battery Park,manhattan
...,...,...
100,Woodlawn Cemetery,bronx
101,Woolworth Building,manhattan
102,World Trade Center site,manhattan
103,World Financial Center,manhattan


In [41]:
# We achieve our clean dataframe:
df_NYmonuments["neighbourhood"].value_counts(dropna=False)

manhattan        70
brooklyn          8
bronx             7
queens            6
other             1
staten island     1
Name: neighbourhood, dtype: int64

In [42]:
#And we export our datas to be able to use them while looking for visualization
df_NYmonuments.to_csv("output/df_NYmonuments_output.csv", index = False)